In [ ]:
!pip3 install flotorch-eval crewai plotly opentelemetry-api opentelemetry-sdk pandas ragas duckduckgo-search openlit langchain_aws


  Using cached crewai-0.126.0-py3-none-any.whl.metadata (35 kB)
  Using cached duckduckgo_search-8.0.2-py3-none-any.whl.metadata (16 kB)
  Using cached openlit-1.33.23-py3-none-any.whl.metadata (23 kB)
  Using cached langchain_aws-0.2.24-py3-none-any.whl.metadata (3.2 kB)
  Using cached auth0_python-4.9.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached chromadb-1.0.12-cp39-abi3-win_amd64.whl.metadata (7.0 kB)
  Using cached instructor-1.8.3-py3-none-any.whl.metadata (24 kB)
  Using cached litellm-1.68.0-py3-none-any.whl.metadata (36 kB)
  Using cached onnxruntime-1.22.0-cp312-cp312-win_amd64.whl.metadata (5.0 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_http-1.34.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached pdfplumber-0.11.6-py3-none-any.whl.metadata (42 kB)
  Using cached pyvis-0.3.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using 

In [1]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../..")))

In [2]:
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from opentelemetry.sdk.trace.export.in_memory_span_exporter import InMemorySpanExporter


# Create an in-memory span exporter
memory_exporter = InMemorySpanExporter()
span_processor = SimpleSpanProcessor(memory_exporter)

# Set up the tracer provider and add the span processor
tracer_provider = TracerProvider()
tracer_provider.add_span_processor(span_processor)
trace.set_tracer_provider(tracer_provider)

In [3]:
import openlit

# Initialize OpenLit - this will automatically instrument CrewAI when it's imported
openlit.init()

Overriding of current TracerProvider is not allowed
Overriding of current TracerProvider is not allowed


{
    "resource_metrics": [
        {
            "resource": {
                "attributes": {
                    "telemetry.sdk.language": "python",
                    "telemetry.sdk.name": "openlit",
                    "telemetry.sdk.version": "1.34.0",
                    "service.name": "default",
                    "deployment.environment": "default"
                },
                "schema_url": ""
            },
            "scope_metrics": [
                {
                    "scope": {
                        "name": "openlit.otel.metrics",
                        "version": "0.1.0",
                        "schema_url": "",
                        "attributes": null
                    },
                    "metrics": [
                        {
                            "name": "gen_ai.client.token.usage",
                            "description": "Measures number of input and output tokens used",
                            "unit": "{token}",
                 

In [4]:
from crewai.tools import tool
from duckduckgo_search import DDGS

@tool('DuckDuckGoSearch')
def search_tool(search_query: str):
    """Search the web for information on a given topic"""
    return DDGS().text(search_query, max_results=5)

@tool('SalesforceIntegration')
def salesforce_tool(soql_query: str):
    """Call Salesforce API to get data"""
    return "Salesforce Integration"

from crewai import LLM

model = LLM(
    # model="sagemaker/INSERT ENDPOINT NAME",
    model="bedrock/us.amazon.nova-pro-v1:0",
    temperature=0.7, max_tokens=4*1024,
)

from crewai import Agent, Task, Crew


writer = Agent(
        role="Writer",
        goal="You make GenAI concepts understandable for newbies exploring GenAI on AWS",
        backstory="You're an expert in writing crisp summaries about GenAI on AWS.",
        tools=[search_tool],
        llm=model
    )

task = Task(description=("What is {topic}?"),
            expected_output=("Compose a short summary that includes the answer."),
            agent=writer)

crew = Crew(
  agents=[writer],
  tasks=[task],
  share_crew=False
)

result = crew.kickoff({"topic": "AWS Bedrock"})
print(result)



{
    "name": "Crew Created",
    "context": {
        "trace_id": "0xb2e35d5c6b64b19f75209257f458e98e",
        "span_id": "0xc7a954c72f3c603a",
        "trace_state": "[]"
    },
    "kind": "SpanKind.INTERNAL",
    "parent_id": null,
    "start_time": "2025-06-10T17:24:54.076259Z",
    "end_time": "2025-06-10T17:24:54.079468Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "crewai_version": "0.126.0",
        "python_version": "3.12.10",
        "crew_key": "a7f38a97312c2a08d7161334f446c413",
        "crew_id": "67dd5643-66dd-4780-9b15-d5e8d93512af",
        "crew_process": "sequential",
        "crew_memory": false,
        "crew_number_of_tasks": 1,
        "crew_number_of_agents": 1,
        "crew_fingerprint": "c67aa107-7e91-46c5-80c6-4fa6ca8d3560",
        "crew_fingerprint_created_at": "2025-06-10T22:54:54.058123",
        "crew_agents": "[{\"key\": \"18e63413ba6e2f4d81ec74e7660d93d9\", \"id\": \"c40f94e2-df88-4db7-827d-2f74fae473c8\", \"role\"

In [5]:
spans = memory_exporter.get_finished_spans()

print("Number of spans:", len(spans))

Number of spans: 7


In [6]:
from langchain_aws import ChatBedrockConverse
from ragas.llms import LangchainLLMWrapper

region = "us-east-1"

bedrock_model = ChatBedrockConverse(
    region_name=region,
    endpoint_url=f"https://bedrock-runtime.us-east-1.amazonaws.com",
    model_id="us.amazon.nova-micro-v1:0"
)

llm = LangchainLLMWrapper(bedrock_model)



c:\Projects\flotorch-eval-proj\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from flotorch_eval.agent_eval.core.converter import TraceConverter

def create_trajectory(spans):
    converter = TraceConverter()
    trajectory = converter.from_spans(spans)
    return trajectory


In [9]:
trajectory = create_trajectory(spans)
trajectory

Trajectory(trace_id='b2e35d5c6b64b19f75209257f458e98e', messages=[Message(role='user', content='What is AWS Bedrock?', tool_calls=[], timestamp=datetime.datetime(2025, 6, 10, 22, 54, 54, 119248)), Message(role='assistant', content='', tool_calls=[ToolCall(name='DuckDuckGoSearch', arguments={'search_query': 'AWS Bedrock'}, output=None, timestamp=datetime.datetime(2025, 6, 10, 22, 54, 54, 119248))], timestamp=datetime.datetime(2025, 6, 10, 22, 54, 54, 119248)), Message(role='tool', content='[{\'title\': \'Amazon Bedrock\', \'href\': \'https://aws.amazon.com/bedrock/\', \'body\': \'Amazon Bedrock. The easiest way to build and scale generative AI applications with foundation models\'}, {\'title\': \'What is Amazon Bedrock? - Amazon Bedrock - docs.aws.amazon.com\', \'href\': \'https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-bedrock.html\', \'body\': \'Amazon Bedrock lets you use and customize high-performing AI models from leading companies and Amazon through a unified API. You

In [10]:
from flotorch_eval.agent_eval.core.evaluator import Evaluator
from flotorch_eval.agent_eval.metrics.langchain_metrics import (
    TrajectoryEvalWithLLMMetric,
    TrajectoryEvalWithoutLLMMetric,
)
from flotorch_eval.agent_eval.metrics.ragas_metrics import (
    AgentGoalAccuracyMetric,
    ToolCallAccuracyMetric,
)
from flotorch_eval.agent_eval.metrics.base import MetricConfig
from flotorch_eval.agent_eval.metrics.usage_metrics import UsageMetric

evaluator = Evaluator([
    ToolCallAccuracyMetric(),
    AgentGoalAccuracyMetric(llm=llm,config=MetricConfig(
        metric_params={
            "reference_answer": "Amazon Bedrock is a fully managed service that makes it easy to use foundation models from third-party providers and Amazon. It allows users to build generative AI applications with a choice of foundation models from different AI companies, using a single API. Users can customize these models with their data, orchestrate multistep tasks, trace reasoning, and apply guardrails for responsible AI. Additionally, Amazon Bedrock enables the creation of generative AI workflows by connecting its features with other AWS services."  # optional
        }
    )),
    TrajectoryEvalWithLLMMetric(llm = bedrock_model,config=MetricConfig(
        metric_params={
            "reference_outputs": [
                {"role": "user", "content": "What is AWS Bedrock?"},
                {
                    "role": "assistant",
                    "content": "To compose a poem about Amazon Bedrock, I first need to gather information about what Amazon Bedrock is. I will use the available tool to search for this information.",
                    "tool_calls": [
                        {
                            "function": {
                                "name": "Search the web for information on a given topic",
                                "arguments": "{\"search_query\": \"Amazon Bedrock\"}"
                            }
                        }
                    ]
                },
                {"role": "tool", "content": "{\"searchParameters\": {\"q\": \"Amazon Bedrock\", \"type\": \"search\", \"num\": 5, \"engine\": \"google\"}, \"organic\": [{\"title\": \"Amazon Bedrock - Generative AI - AWS\", \"link\": \"https://aws.amazon.com/bedrock/\", \"snippet\": \"Amazon Bedrock Data Automation streamlines the generation of valuable insights from unstructured multimodal content such as documents, images, audio, and videos ...\", \"position\": 1, \"sitelinks\": [{\"title\": \"Amazon Bedrock\", \"link\": \"https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-bedrock.html\"}, {\"title\": \"Amazon Bedrock Pricing\", \"link\": \"https://aws.amazon.com/bedrock/pricing/\"}, {\"title\": \"Amazon Bedrock Documentation\", \"link\": \"https://docs.aws.amazon.com/bedrock/\"}, {\"title\": \"Amazon Bedrock FAQs\", \"link\": \"https://aws.amazon.com/bedrock/faqs/\"}, {\"title\": \"Amazon Bedrock Agents\", \"link\": \"https://aws.amazon.com/bedrock/agents/\"}]}, {\"title\": \"Getting Started with Amazon Bedrock - AWS\", \"link\": \"https://aws.amazon.com/awstv/watch/6ff4cd6fa97/\", \"snippet\": \"So check the region that you're currently in, make sure it's a region that's supported by Bedrock. Then I'm gonna scroll to the bottom of this ...\", \"position\": 2}], \"relatedSearches\": [{\"query\": \"Amazon Bedrock pricing\"}, {\"query\": \"Amazon Bedrock documentation\"}, {\"query\": \"Amazon Bedrock Claude\"}, {\"query\": \"Amazon Bedrock logo\"}, {\"query\": \"Amazon Bedrock DeepSeek\"}], \"credits\": 1}"},
                {"role": "assistant", "content": "Based on the observation, I have learned that mazon Bedrock is a fully managed service that makes it easy to use foundation models from third-party providers and Amazon. It allows users to build generative AI applications with a choice of foundation models from different AI companies, using a single API. Users can customize these models with their data, orchestrate multistep tasks, trace reasoning, and apply guardrails for responsible AI. Additionally, Amazon Bedrock enables the creation of generative AI workflows by connecting its features with other AWS services."}
            ]
        }
    )),
    UsageMetric(config=MetricConfig(
        metric_params={"aws_region": "us-east-1"}
    ))
])

# Evaluate trajectory
results = await evaluator.evaluate(trajectory)



Reading CSV file from: c:\Projects\flotorch-eval-proj\flotorch_eval\data\bedrock_limits_small.csv
{'user': {'role': 'user', 'content': '{\'text\': \'Given an agentic workflow comprised of Human, AI and Tools, identify the user_goal (the task or objective the user wants to achieve) and the end_state (the final outcome or result of the workflow).\\nPlease return the output in a JSON format that complies with the following schema as specified in JSON Schema:\\n{"properties": {"user_goal": {"description": "The task or objective the user wants to achieve.", "title": "User Goal", "type": "string"}, "end_state": {"description": "The final outcome or result of the workflow.", "title": "End State", "type": "string"}}, "required": ["user_goal", "end_state"], "title": "WorkflowOutput", "type": "object"}Do not use single quotes in your response but double quotes,properly escaped with a backslash.\\n\\n--------EXAMPLES-----------\\nExample 1\\nInput: {\\n    "workflow": "\\\\n            Human: Hey

Invalid type NoneType for attribute 'gen_ai.completion' value. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or a sequence of those types


{'user': {'role': 'user', 'content': '{\'text\': \'You are an expert data labeler.\\nYour task is to grade the accuracy of an AI agent\\\'s internal trajectory.\\n\\n<Rubric>\\n  An accurate trajectory:\\n  - Makes logical sense between steps\\n  - Shows clear progression\\n  - Is relatively efficient, though it does not need to be perfectly efficient\\n  - Is semantically equivalent to the provided reference trajectory\\n</Rubric>\\n\\nBased on the following reference trajectory:\\n\\n<reference_trajectory>\\n<user>\\nWhat is AWS Bedrock?\\n</user>\\n\\n<assistant>\\nTo compose a poem about Amazon Bedrock, I first need to gather information about what Amazon Bedrock is. I will use the available tool to search for this information.\\n<tool_call>\\n<name>Search the web for information on a given topic</name>\\n<arguments>{"search_query": "Amazon Bedrock"}</arguments>\\n</tool_call>\\n</assistant>\\n\\n<tool>\\n{"searchParameters": {"q": "Amazon Bedrock", "type": "search", "num": 5, "eng

Invalid type dict in attribute 'gen_ai.completion' value sequence. Expected one of ['bool', 'str', 'bytes', 'int', 'float'] or None


{
    "name": "chat us.amazon.nova-micro-v1:0",
    "context": {
        "trace_id": "0x3a9f13110c95b2f7ed66bc5d41cbfe65",
        "span_id": "0x619c28fa2463790d",
        "trace_state": "[]"
    },
    "kind": "SpanKind.CLIENT",
    "parent_id": null,
    "start_time": "2025-06-10T17:25:29.012838Z",
    "end_time": "2025-06-10T17:25:30.466716Z",
    "status": {
        "status_code": "OK"
    },
    "attributes": {
        "telemetry.sdk.name": "openlit",
        "gen_ai.operation.name": "chat",
        "gen_ai.system": "langchain",
        "gen_ai.request.model": "us.amazon.nova-micro-v1:0",
        "gen_ai.response.model": "us.amazon.nova-micro-v1:0",
        "gen_ai.request.temperature": "1e-08",
        "gen_ai.request.top_k": "1",
        "gen_ai.request.top_p": "None",
        "gen_ai.usage.input_tokens": 2691,
        "gen_ai.usage.output_tokens": 946,
        "server.address": "NOT_FOUND",
        "server.port": "NOT_FOUND",
        "deployment.environment": "default",
       

In [11]:
results.scores

[MetricResult(name='tool_call_accuracy', score=1.0, details={'evaluation_type': 'tool_call_accuracy'}),
 MetricResult(name='agent_goal_accuracy', score=0.0, details={}),
 MetricResult(name='trajectory_eval_with_llm', score=1.0, details={'comment': "The trajectory accurately answers the question about AWS Bedrock by defining it as a fully managed service from AWS. It shows logical sense by using a tool to search for information and presenting a coherent response. It shows clear progression from the question to the search call to the final informative answer. It is relatively efficient though it doesn't need to be perfectly efficient. It is semantically equivalent to the reference trajectory. Thus, the score should be: True.", 'has_reference': True, 'raw_score': True}),
 MetricResult(name='usage_summary', score=0.001562, details={'total_cost': 0.001562, 'average_cost_per_call': 0.000781, 'cost_breakdown': [{'model': 'us.amazon.nova-pro-v1:0', 'input_tokens': 305, 'output_tokens': 20, 'co